In [ ]:
GOOGLE_API_KEY = "YOUR_API_KEY_HERE"

In [2]:
import os
import json
import google.generativeai as genai
from pptx import Presentation
from pptx.util import Inches
from duckduckgo_search import DDGS

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google API Key configured successfully.")
else:
    print("Please paste your GOOGLE_API_KEY.")

Google API Key configured successfully.


In [3]:
def get_web_search_results(topic, num_results=5):

    print(f"Performing web search for '{topic}'...")
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(topic, max_results=num_results))
            if not results:
                return "No search results found."
            search_context = "\n\n".join(
                [f"Title: {r['title']}\nLink: {r['href']}\nSnippet: {r['body']}" for r in results]
            )
            print("Web search completed.")
            return search_context
    except Exception as e:
        print(f"Error during web search: {e}")
        return "Web search failed."

In [4]:
import requests
from bs4 import BeautifulSoup

def get_web_search_results(topic, num_results=3):

    print(f"Performing web search for '{topic}'...")
    search_context = ""
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(topic, max_results=num_results))
            if not results:
                return "No search results found."

            for result in results:
                print(f"  - Scraping content from: {result['href']}")
                try:
                    response = requests.get(result['href'], timeout=5)
                    response.raise_for_status() # Raise an error 

                    soup = BeautifulSoup(response.content, 'html.parser')
                    paragraphs = soup.find_all('p')
                    page_text = "\n".join([p.get_text() for p in paragraphs])

                    search_context += f"Title: {result['title']}\nLink: {result['href']}\nContent: {page_text[:1500]}\n\n---\n\n" 

                except Exception as e:
                    print(f"    - Could not scrape {result['href']}: {e}")
                    search_context += f"Title: {result['title']}\nLink: {result['href']}\nSnippet: {result['body']}\n\n---\n\n"

            print("Web search and scraping completed.")
            return search_context

    except Exception as e:
        print(f"Error during web search: {e}")
        return "Web search failed."

In [5]:
def generate_presentation_content(topic, search_results):

    print("Generating presentation content for a 7-slide structure...")

    model = genai.GenerativeModel(model_name="gemini-1.5-flash")

    prompt = f"""
    Act as an expert researcher. Your task is to generate the content for a 7-slide presentation on the topic: "{topic}".
    Use your internal knowledge and the provided web search results to create the content.

    **Web Search Results:**
    ---
    {search_results}
    ---

    The output MUST be a single, valid JSON object. Do not include any text before or after the JSON.

    The JSON structure must be as follows:
    {{
      "title_slide": {{ "title": "A short, engaging title", "subtitle": "A brief, descriptive subtitle" }},
      "overview_slide": {{ "title": "Overview", "points": ["Point 1", "Point 2", "Point 3"] }},
      "key_point_slides": [
        {{ "title": "Key Point 1", "points": ["A detailed bullet point.", "Another detailed bullet point.", "A final detailed bullet point."] }},
        {{ "title": "Key Point 2", "points": ["A detailed bullet point.", "Another detailed bullet point.", "A final detailed bullet point."] }},
        {{ "title": "Key Point 3", "points": ["A detailed bullet point.", "Another detailed bullet point.", "A final detailed bullet point."] }},
        {{ "title": "Key Point 4", "points": ["A detailed bullet point.", "Another detailed bullet point.", "A final detailed bullet point."] }}
      ],
      "conclusion_slide": {{ "title": "Conclusion", "points": ["Summary of key takeaways.", "Final concluding thought."] }}
    }}

    **IMPORTANT INSTRUCTION:** For each of the 4 "key_point_slides", ensure that the combined text of all its "points" is at least 100 words. Be descriptive and elaborate on each bullet point.
    """
    try:
        response = model.generate_content(prompt)
        json_text = response.text.strip().replace("```json", "").replace("```", "")
        content = json.loads(json_text)
        print("LLM content generated and parsed successfully.")
        return content
    except Exception as e:
        print(f"Error generating or parsing LLM content: {e}")
        print("LLM Response Text:", response.text if 'response' in locals() else "No response")
        return None

In [6]:
from pptx import Presentation
from pptx.util import Inches, Pt 

def create_powerpoint_presentation(content, topic):

    print("Creating PowerPoint presentation...")
    prs = Presentation()

    title_slide_layout = prs.slide_layouts[0]
    content_slide_layout = prs.slide_layouts[1]

    def add_content_slide(title_text, points_list):
        slide = prs.slides.add_slide(content_slide_layout)
        slide.shapes.title.text = title_text
        
        body_placeholder = slide.placeholders[1]
        text_frame = body_placeholder.text_frame
        text_frame.clear() 

        text_frame.auto_size = True
        text_frame.word_wrap = True

        if points_list:
            p = text_frame.paragraphs[0]
            p.text = points_list[0]
            p.level = 0
            p.font.size = Pt(18)

            for point_text in points_list[1:]:
                p = text_frame.add_paragraph()
                p.text = point_text
                p.level = 0
                p.font.size = Pt(18)

    title_info = content["title_slide"]
    slide = prs.slides.add_slide(title_slide_layout)
    slide.shapes.title.text = title_info["title"]
    
    subtitle_shape = slide.placeholders[1]
    subtitle_shape.text = title_info["subtitle"]
    subtitle_shape.text_frame.auto_size = True
    subtitle_shape.text_frame.word_wrap = True


    overview_info = content["overview_slide"]
    add_content_slide(overview_info["title"], overview_info["points"])

    for key_point_slide in content["key_point_slides"]:
        add_content_slide(key_point_slide["title"], key_point_slide["points"])

    conclusion_info = content["conclusion_slide"]
    add_content_slide(conclusion_info["title"], conclusion_info["points"])

    filename = f"{topic.replace(' ', '_')}_Presentation.pptx"
    prs.save(filename)
    print(f"✅ Presentation saved as '{filename}'")
    return filename

## MAIN function

In [11]:
def main():

    user_topic = input("Enter the topic for your presentation: ")
    if not user_topic:
        print("Topic cannot be empty.")
        return

    # 1. Perform web search
    search_results = get_web_search_results(user_topic)
    if "failed" in search_results or "No search results" in search_results:
        return

    # 2. Generate content using LLM
    presentation_content = generate_presentation_content(user_topic, search_results)
    if not presentation_content:
        print("Halting process due to content generation failure.")
        return

    # 3. Create PowerPoint file
    create_powerpoint_presentation(presentation_content, user_topic)
    print("\nAll done! Your presentation is ready.")

# Run the main function
if __name__ == "__main__":
    main()

Enter the topic for your presentation:  corruption in india


Performing web search for 'corruption in india'...


C:\Users\kanis\AppData\Local\Temp\ipykernel_26612\945892756.py:9: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


  - Scraping content from: https://en.wikipedia.org/wiki/Corruption_in_India
    - Could not scrape https://en.wikipedia.org/wiki/Corruption_in_India: 403 Client Error: Forbidden for url: https://en.wikipedia.org/wiki/Corruption_in_India
  - Scraping content from: https://www.drishtiias.com/to-the-points/paper4/corruption-in-india
  - Scraping content from: https://www.insightsonindia.com/2025/06/09/corruption/
Web search and scraping completed.
Generating presentation content for a 7-slide structure...
LLM content generated and parsed successfully.
Creating PowerPoint presentation...
✅ Presentation saved as 'corruption_in_india_Presentation.pptx'

All done! Your presentation is ready.


#### How snippets look for DDGS

In [8]:
from duckduckgo_search import DDGS
import textwrap

def web_search_results(topic, num_results=3):

    print(f"Performing web search for '{topic}'...")
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(topic, max_results=num_results))
            if not results:
                return "No search results found."
            
            search_context = "\n\n---\n\n".join(
                [f"Title: {r['title']}\nLink: {r['href']}\nSnippet: {r['body']}" for r in results]
            )
            print("Web search completed.")
            return search_context
    except Exception as e:
        print(f"Error during web search: {e}")
        return "Web search failed."


test_topic = "ganesh chaturty"

search_results = web_search_results(test_topic)

print("\n" + "="*50)
print("   FINAL search_results")
print("="*50 + "\n")
print(search_results)

Performing web search for 'ganesh chaturty'...


C:\Users\kanis\AppData\Local\Temp\ipykernel_27820\2649384116.py:8: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Web search completed.

   FINAL search_results

Title: Ganesh Chaturthi - Wikipedia
Link: https://en.m.wikipedia.org/wiki/Ganesh_Chaturthi
Snippet: Ganesh Chaturthi (ISO: Gaṇeśa Caturthī) (transl. Ganesh Festival or the Birthday of Lord Ganesh), also known as Vinayaka Chaturthi (Vināyaka Caturthī) or Vinayaka Chavithi (Vināyaka Cavithī) or …

---

Title: Ganesh Chaturthi | Hinduism, Festival, Observances, Significance ...
Link: https://www.britannica.com/topic/Ganesh-Chaturthi
Snippet: 6 days ago · Ganesh Chaturthi is a popular festival in Hinduism marking the birth of the elephant-headed deity Ganesha, the god of prosperity, wisdom, and removal of obstacles. It begins on the …

---

Title: Ganesh Chaturthi 2025: Date, Puja Timings, Rituals, and …
Link: https://caleidoscope.in/art-culture/ganesh-chaturthi-date-puja-timings-rituals-and-significance
Snippet: 2 days ago · Ganesh Chaturthi 2025 is on Aug 27. Explore puja muhurat, visarjan date, rituals, and significance of this auspicious 

#### IF we use request instead of snippet to get more content

In [13]:
import requests
from bs4 import BeautifulSoup

def get_web_search_results(topic, num_results=3):

    print(f"Performing web search for '{topic}'...")
    search_context = ""
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(topic, max_results=num_results))
            if not results:
                return "No search results found."

            for result in results:
                print(f"  - Scraping content from: {result['href']}")
                try:
                    response = requests.get(result['href'], timeout=5)
                    response.raise_for_status() # Raise an error 

                    soup = BeautifulSoup(response.content, 'html.parser')
                    paragraphs = soup.find_all('p')
                    page_text = "\n".join([p.get_text() for p in paragraphs])

                    search_context += f"Title: {result['title']}\nLink: {result['href']}\nContent: {page_text[:1500]}\n\n---\n\n" 

                except Exception as e:
                    print(f"    - Could not scrape {result['href']}: {e}")
                    search_context += f"Title: {result['title']}\nLink: {result['href']}\nSnippet: {result['body']}\n\n---\n\n"

            print("Web search and scraping completed.")
            return search_context

    except Exception as e:
        print(f"Error during web search: {e}")
        return "Web search failed."
        
test_topic = "ganesh chaturty"

search_results = web_search_results(test_topic)

print("\n" + "="*50)
print("   FINAL search_results")
print("="*50 + "\n")
print(search_results)

Performing web search for 'ganesh chaturty'...


C:\Users\kanis\AppData\Local\Temp\ipykernel_27820\2696708899.py:23: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


  - Scraping content from: https://en.m.wikipedia.org/wiki/Ganesh_Chaturthi
  - Scraping content from: https://www.britannica.com/topic/Ganesh-Chaturthi
  - Scraping content from: https://caleidoscope.in/art-culture/ganesh-chaturthi-date-puja-timings-rituals-and-significance
Web search and scraping completed.

   FINAL search_results

Title: Ganesh Chaturthi - Wikipedia
Link: https://en.m.wikipedia.org/wiki/Ganesh_Chaturthi
Content: 

Ganesh Chaturthi (ISO: Gaṇeśa Caturthī) (transl. Ganesh Festival or the Birthday of Lord Ganesh), also known as Vinayaka Chaturthi (Vināyaka Caturthī) or Vinayaka Chavithi (Vināyaka Cavithī) or Vinayagar Chaturthi (Vināyagar Caturthī), is a Hindu festival celebrating the birthday of Hindu deity Ganesha.[1] The festival is marked with the installation of Ganesha's murtis (devotional representations of a deity) privately in homes and publicly on elaborate pandals (temporary stages). Observances include chanting of Vedic hymns and Hindu texts, such as prayer

#### Instead of request, we use SELENIUM

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from duckduckgo_search import DDGS

def web_search_results(topic, num_results=3):

    print(f"Performing web search for '{topic}'...")
    search_context = ""

    chrome_options = Options()
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    driver = None
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        with DDGS() as ddgs:
            results = list(ddgs.text(topic, max_results=num_results))
            if not results:
                return "No search results found."

        for result in results:
            print(f"  - Scraping content from: {result['href']}")
            try:
                driver.get(result['href'])
                
                paragraphs = driver.find_elements(By.TAG_NAME, 'p')
                
                page_text = "\n".join([p.text for p in paragraphs])

                search_context += f"Title: {result['title']}\nLink: {result['href']}\nContent: {page_text[:1500]}\n\n---\n\n"

            except Exception as e:
                print(f"    - Could not scrape {result['href']}: {e}")
                search_context += f"Title: {result['title']}\nLink: {result['href']}\nSnippet: {result['body']}\n\n---\n\n"

        print("Web search and scraping completed.")
        return search_context

    except Exception as e:
        print(f"Error during web search or Selenium setup: {e}")
        return "Web search failed."
    finally:
        if driver:
            driver.quit()


test_topic = "ganesh chaturty"

search_results = web_search_results(test_topic)

print("\n" + "="*50)
print("   FINAL search_results")
print("="*50 + "\n")
print(search_results)

Performing web search for 'ganesh chaturty'...


C:\Users\kanis\AppData\Local\Temp\ipykernel_27820\2696708899.py:23: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


  - Scraping content from: https://en.m.wikipedia.org/wiki/Ganesh_Chaturthi
  - Scraping content from: https://www.britannica.com/topic/Ganesh-Chaturthi
  - Scraping content from: https://caleidoscope.in/art-culture/ganesh-chaturthi-date-puja-timings-rituals-and-significance
Web search and scraping completed.

   FINAL search_results

Title: Ganesh Chaturthi - Wikipedia
Link: https://en.m.wikipedia.org/wiki/Ganesh_Chaturthi
Content: 

Ganesh Chaturthi (ISO: Gaṇeśa Caturthī) (transl. Ganesh Festival or the Birthday of Lord Ganesh), also known as Vinayaka Chaturthi (Vināyaka Caturthī) or Vinayaka Chavithi (Vināyaka Cavithī) or Vinayagar Chaturthi (Vināyagar Caturthī), is a Hindu festival celebrating the birthday of Hindu deity Ganesha.[1] The festival is marked with the installation of Ganesha's murtis (devotional representations of a deity) privately in homes and publicly on elaborate pandals (temporary stages). Observances include chanting of Vedic hymns and Hindu texts, such as prayer

#### Doing seach in CHROME the get data from DDGS

In [11]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

def search_and_scrape_with_chrome(topic, num_results=3):

    print(f"Starting Selenium-only search for '{topic}'...")
    search_context = ""

    chrome_options = Options()
    chrome_options.add_argument("--start-maximized") 
    
    driver = None
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        print("  - Opening browser and navigating to DuckDuckGo...")
        driver.get("https://duckduckgo.com/")
        time.sleep(2) # Wait a moment for the page to load

        search_box = driver.find_element(By.ID, "searchbox_input")
        search_box.send_keys(topic)
        search_box.send_keys(Keys.RETURN)
        print(f"  - Searching for '{topic}'...")
        time.sleep(2) # Wait for search results to load

        links = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='result-title-a']")
        urls_to_scrape = [link.get_attribute('href') for link in links[:num_results]]
        titles = [link.text for link in links[:num_results]]
        
        if not urls_to_scrape:
            return "No search results found on the page."

        for i, url in enumerate(urls_to_scrape):
            print(f"  - Scraping content from: {url}")
            try:
                driver.get(url)
                time.sleep(2) 
                
                paragraphs = driver.find_elements(By.TAG_NAME, 'p')
                page_text = "\n".join([p.text for p in paragraphs])

                search_context += f"Title: {titles[i]}\nLink: {url}\nContent: {page_text[:1000]}...\n\n---\n\n"

            except Exception as e:
                print(f"    - Could not scrape {url}: {e}")

        print("Web search and scraping completed.")
        return search_context

    except Exception as e:
        print(f"An error occurred: {e}")
        return "Process failed."
    finally:
        if driver:
            print(" closing the browser in 5 seconds...")
            time.sleep(5)
            driver.quit()

dummy_topic = "Benefits of learning a new language"
final_content = search_and_scrape_with_chrome(dummy_topic)

print("\n" + "="*50)
print("   FINAL SCRAPED CONTENT")
print("="*50 + "\n")
print(final_content)

Starting Selenium-only search for 'Benefits of learning a new language'...
  - Opening browser and navigating to DuckDuckGo...
  - Searching for 'Benefits of learning a new language'...
  - Scraping content from: https://gentwenty.com/13-benefits-of-learning-a-new-language/
  - Scraping content from: https://potomac.edu/benefits-of-learning-a-second-language/
  - Scraping content from: https://www.cambridge.org/elt/blog/2022/04/29/learning-language-changes-your-brain/
Web search and scraping completed.
 closing the browser in 5 seconds...

   FINAL SCRAPED CONTENT

Title: 13 Benefits of Learning a New Language and Why It's Worth ... - GenTwenty
Link: https://gentwenty.com/13-benefits-of-learning-a-new-language/
Content: By:
Author
Mia Barnes
Posted on
Last updated: July 23, 2025
Categories
Personal Growth, Self Development
In an ever-connected world, learning a new language offers more than the ability to communicate—it opens doors to personal growth, cultural understanding and expande

#### Using chrome for every search

In [12]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

def search_and_scrape_with_chrome(topic, num_results=3):

    print(f"Starting Selenium-only search for '{topic}'...")
    search_context = ""

    chrome_options = Options()
    chrome_options.add_argument("--start-maximized") 
    chrome_options.add_experimental_option('prefs', {'profile.default_content_setting_values.cookies': 1})
    
    driver = None
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        print("  - Opening browser and navigating to Google...")
        driver.get("https://www.google.com")
        time.sleep(2) # Wait a moment for the page to load

        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys(topic)
        search_box.send_keys(Keys.RETURN)
        print(f"  - Searching for '{topic}'...")
        time.sleep(3) # Wait for search results to load

        # --- 2. Get the URLs from the results page ---
        # This finds the main result links (h3 tags within divs) on the Google results page
        links = driver.find_elements(By.CSS_SELECTOR, "div.g a")
        
        urls_to_scrape = []
        for link in links:
            href = link.get_attribute('href')
            if href and href.startswith('http'):
                urls_to_scrape.append(href)
            if len(urls_to_scrape) >= num_results:
                break
        
        if not urls_to_scrape:
            return "No search results found on the page."

        for url in urls_to_scrape:
            print(f"  - Scraping content from: {url}")
            try:
                driver.get(url)
                time.sleep(3) 
                
                title = driver.title

                paragraphs = driver.find_elements(By.TAG_NAME, 'p')
                
                page_text = "\n".join([p.text for p in paragraphs])

                search_context += f"Title: {title}\nLink: {url}\nContent: {page_text[:1000]}...\n\n---\n\n"

            except Exception as e:
                print(f"    - Could not scrape {url}: {e}")

        print("Web search and scraping completed.")
        return search_context

    except Exception as e:
        print(f"An error occurred: {e}")
        return "Process failed."
    finally:
        if driver:
            print(" closing the browser in 5 seconds...")
            time.sleep(5)
            driver.quit()

dummy_topic = "Benefits of learning a new language"
final_content = search_and_scrape_with_chrome(dummy_topic)

print("\n" + "="*50)
print("   FINAL SCRAPED CONTENT")
print("="*50 + "\n")
print(final_content)

Starting Selenium-only search for 'Benefits of learning a new language'...
  - Opening browser and navigating to Google...
  - Searching for 'Benefits of learning a new language'...
 closing the browser in 5 seconds...

   FINAL SCRAPED CONTENT

No search results found on the page.
